In [251]:
%load_ext autoreload
%autoreload 2

../data/models/
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [252]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
from Constants import *
from Preprocessing import *
from Models import *
pd.set_option('display.max_rows', 200)



In [192]:
pd.read_csv('../data/digital_twin_ln_data.csv')

,id,cluster,1A1B,1A6,1B2A,1B3,2A2B,2A3,2B5A,34,...,1A,1B,2A,2B,3,4,5A,5B,6,RPLN
0,1,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,5089,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5088,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5087,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5085,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,1033726,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1176,1033946,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1177,1035327,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1178,1035635,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [195]:
pd.read_csv('../data/digital_twin_ln_monograms.csv')

,id,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,...,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
0,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,10201,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
532,10202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
533,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
534,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [262]:
data = DTDataset(use_smote=False,smote_ids = Const.stratified_train_ids,ln_data_file = '../data/digital_twin_ln_monograms.csv')
data.processed_df#.shape, len(data.processed_df.index.unique())

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,hpv,age,packs_per_year,smoking_status,gender,Aspiration rate Pre-therapy,total_dose,dose_fraction,bilateral,cc_none,...,4_ipsi,4_contra,5A_ipsi,5A_contra,5B_ipsi,5B_contra,6_ipsi,6_contra,RPLN_ipsi,RPLN_contra
id,,,,,,,,,,,,,,,,,,,,,
3,1,55.969444,0.0,0.0,1,0,66.00,2.200000,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,20.950000,38.0,1.0,1,0,72.00,1.800000,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,69.930556,35.0,1.0,0,1,70.00,2.121212,True,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,72.319444,0.0,1.0,1,0,70.00,2.121212,False,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,59.730556,0.0,0.0,1,0,66.00,2.200000,False,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10201,1,49.566667,30.0,1.0,1,0,70.00,2.121212,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10202,0,48.705556,30.0,1.0,1,0,72.00,1.714286,False,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10203,1,77.116667,0.0,0.0,1,0,70.00,2.333333,False,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [263]:
temp = data.processed_df[data.processed_df['DLT_Nephrological'].apply(lambda x: x>0)]
temp[[c for c in data.processed_df if 'DLT' in c]]

,DLT (Y/N),DLT_Dermatological,DLT_Neurological,DLT_Gastrointestinal,DLT_Hematological,DLT_Nephrological,DLT_Vascular,DLT_Infection (Pneumonia),DLT_Grade,DLT_Other,DLT_Dermatological 2,DLT_Neurological 2,DLT_Gastrointestinal 2,DLT_Hematological 2,DLT_Nephrological 2,DLT_Vascular 2,DLT_Infection (Pneumonia) 2,DLT_Other 2
id,,,,,,,,,,,,,,,,,,
200,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10136,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
model_dir = Const.data_dir + '/models/'

tuned_transition_models = [
    'final_transition1_model_state1_input63_dims500,500_dropout0.25,0.5.pt',
    'final_transition2_model_state2_input85_dims100_dropout0.25,0.pt',
    'final_outcome_model_state1_input83_dims1000_dropout0,0.pt'
]
tuned_transition_models = [model_dir + f for f in tuned_transition_models]
Const.tuned_transition_models = tuned_transition_models
Const.tuned_decision_model = model_dir +  'final_decision_model_statedecisions_input132_dims100,100_dropout0.1,0.7.pt'

In [241]:
def get_dt_ids():
    df = load_digital_twin()
    return df.id.values

def get_tt_split(ids=None,use_default_split=True,use_bagging_split=False,resample_training=False):
        if ids is None:
            ids = get_dt_ids()
        #pre-made, stratified by decision and outcome 72:28
        if use_default_split:
            train_ids = Const.stratified_train_ids[:]
            test_ids = Const.stratified_test_ids[:]
        elif use_bagging_split:
            train_ids = np.random.choice(ids,len(ids),replace=True)
            test_ids = [i for i in ids if i not in train_ids]
        else:
            test_ids = ids[0: int(len(ids)*(1-split))]
            train_ids = [i for i in ids if i not in test_ids]

        if resample_training:
            train_ids = np.random.choice(train_ids,len(train_ids),replace=True)
            test_ids = [i for i in ids if i not in train_ids]
        return train_ids,test_ids
    
def df_to_torch(df,ttype  = torch.FloatTensor):
    values = df.values.astype(float)
    values = torch.from_numpy(values)
    return values.type(ttype)

In [242]:
def nllloss(ytrue,ypred):
    #nll loss with argmax added in
    loss = torch.nn.NLLLoss()
    return loss(ypred,ytrue.argmax(axis=1))

def state_loss(ytrue,ypred,weights=[1,1,1,1]):
    pd_loss = nllloss(ytrue[0],ypred[0])*weights[0]
    nd_loss = nllloss(ytrue[1],ypred[1])*weights[1]
    mod_loss = nllloss(ytrue[2],ypred[2])*weights[2]
    loss = pd_loss + nd_loss + mod_loss
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        loss += dlt_loss*weights[3]/ndlt
    return loss

def outcome_loss(ytrue,ypred,weights=[1,1,1]):
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

def mc_metrics(yt,yp,numpy=False,is_dlt=False):
    if not numpy:
        yt = yt .cpu().detach().numpy()
        yp = yp.cpu().detach().numpy()
    #dlt prediction (binary)
    if is_dlt:
        acc = accuracy_score(yt,yp>.5)
        if yt.sum() > 1:
            auc = roc_auc_score(yt,yp)
        else:
            auc=-1
        error = np.mean((yt-yp)**2)
        return {'accuracy': acc, 'mse': error, 'auc': auc}
    #this is a catch for when I se the dlt prediction format (encoded integer ordinal, predict as a categorical and take the argmax)
    elif yt.ndim > 1:
        try:
            bacc = balanced_accuracy_score(yt.argmax(axis=1),yp.argmax(axis=1))
        except:
            bacc = -1
        try:
            roc_micro = roc_auc_score(yt,yp,average='micro')
        except:
            roc_micro=-1
        try:
            roc_macro = roc_auc_score(yt,yp,average='macro')
        except:
            roc_macro = -1
        return {'accuracy': bacc, 'roc_micro': roc_micro,'roc_macro': roc_macro}
    #outcomes (binary)
    else:
        if yp.ndim > 1:
            yp = yp.argmax(axis=1)
        try:
            bacc = accuracy_score(yt,yp)
        except:
            bacc = -1
        try:
            roc = roc_auc_score(yt,yp)
        except:
            roc = -1
        error = np.mean((yt-yp)**2)
        return {'accuracy': bacc, 'mse': error, 'auc': roc}

def state_metrics(ytrue,ypred,numpy=False):
    pd_metrics = mc_metrics(ytrue[0],ypred[0],numpy=numpy)
    nd_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    mod_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    
    dlt_metrics = []
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
    nloss = torch.nn.NLLLoss()
    for i in range(ndlt):
        dm = mc_metrics(dlt_true[:,i],dlt_pred[:,i].view(-1),is_dlt=True)
        dlt_metrics.append(dm)
    dlt_acc =[d['accuracy'] for d in dlt_metrics]
    dlt_error = [d['mse'] for d in dlt_metrics]
    dlt_auc = [d['auc'] for d in dlt_metrics]
    return {'pd': pd_metrics,'nd': nd_metrics,'mod': mod_metrics,'dlts': {'accuracy': dlt_acc,'accuracy_mean': np.mean(dlt_acc),'auc': dlt_auc,'auc_mean': np.mean(dlt_auc)}}
    
def outcome_metrics(ytrue,ypred,numpy=False):
    res = {}
    for i, outcome in enumerate(Const.outcomes):
        metrics = mc_metrics(ytrue[i],ypred[:,i])
        res[outcome] = metrics
    return res


In [44]:
from sklearn.ensemble import RandomForestClassifier

def train_state_rf(model_args={}):
    ids = get_dt_ids()
    
    dataset = DTDataset()

    train_ids = ids[0:int(len(ids)*.7)]
    test_ids = ids[int(len(ids)*.7):]
    
    #most things are multiclass, dlts are several ordinal and outcomes are multiple binary
    xtrain1 = dataset.get_state('baseline',ids=train_ids)
    xtest1 = dataset.get_state('baseline',ids=test_ids)
    
    xtrain2 = dataset.get_input_state(step=2,ids=train_ids)
    xtest2 = dataset.get_input_state(step=2,ids=test_ids)
    
    xtrain3 = dataset.get_input_state(step=3,ids=train_ids)
    xtest3 = dataset.get_input_state(step=3,ids=test_ids)
    
    [pd1_train,nd1_train, mod_train,dlts1_train] = dataset.get_intermediate_outcomes(ids=train_ids)
    [pd2_train,nd2_train, cc_train,dlts2_train] = dataset.get_intermediate_outcomes(step=2,ids=train_ids)
    [pd1_test,nd1_test, mod_test,dlts1_test] = dataset.get_intermediate_outcomes(ids=test_ids)
    [pd2_test,nd2_test, cc_test,dlts2_test] = dataset.get_intermediate_outcomes(step=2,ids=test_ids)
    outcomes_train = dataset.get_state('outcomes',ids=train_ids)
    outcomes_test = dataset.get_state('outcomes',ids=test_ids)
    

    def train_multiclass_rf(xtrain,xtest,ytrain,ytest):
        model = RandomForestClassifier(class_weight='balanced',**model_args).fit(xtrain,ytrain)
        ypred = model.predict(xtest)
        metrics = mc_metrics(ytest.values,ypred,numpy=True)
        return model, metrics
    
    all_metrics = {}
    pd1_model, all_metrics['pd1'] = train_multiclass_rf(xtrain1,xtest1,pd1_train,pd1_test)
    nd1_model, all_metrics['nd1']  = train_multiclass_rf(xtrain1,xtest1,nd1_train,nd1_test)
    mod_model, all_metrics['mod']  = train_multiclass_rf(xtrain1,xtest1,mod_train,mod_test)
    
    return all_metrics

train_state_rf({'max_depth': 5,'n_estimators': 100})

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  


Index(['hpv', 'age', 'packs_per_year', 'smoking_status', 'gender',
       'Aspiration rate Pre-therapy', 'total_dose', 'dose_fraction', '1A1B',
       '1A6',
       ...
       'SD Primary 2', 'CR Nodal 2', 'PR Nodal 2', 'SD Nodal 2',
       'Decision 1 (Induction Chemo) Y/N', 'Decision 2 (CC / RT alone)',
       'Decision 3 Neck Dissection (Y/N)', 'Overall Survival (4 Years)', 'FT',
       'Aspiration rate Post-therapy'],
      dtype='object', length=111)


{'pd1': {'accuracy': 0.36326291079812206,
  'roc_micro': 0.5126910126910126,
  'roc_macro': 0.5150862068965517},
 'nd1': {'accuracy': 0.38838037121789987,
  'roc_micro': 0.5717430717430717,
  'roc_macro': 0.5260869565217391},
 'mod': {'accuracy': 0.16666666666666666,
  'roc_micro': 0.515527950310559,
  'roc_macro': -1}}

In [243]:
 DTDataset().get_state('baseline').T

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


id,3,5,6,7,8,9,10,11,13,14,...,10196,10197,10198,10199,10200,10201,10202,10203,10204,10205
1A_contra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1A_ipsi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1B_contra,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1B_ipsi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2A_contra,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2A_ipsi,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2B_contra,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2B_ipsi,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
3_contra,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3_ipsi,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


In [244]:
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.0001,
                epochs=1000,
                patience=10,
                use_attention=True,
                weights=[1,1,1,10],
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                use_smote=True,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()

    xtrain = dataset.get_input_state(step=state,ids=train_ids)
    xtest = dataset.get_input_state(step=state,ids=test_ids)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids)
    

    if not use_attention:
        model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
            if use_attention:
                model = OutcomeAttentionSimulator(xtrain.shape[1],state=state,**model_args)
            else:
                model = OutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
    else:
        if model is None:
            if use_attention:
                model = EndpointAttentionSimulator(xtrain.shape[1],**model_args)
            else:
                model = EndpointSimulator(xtrain.shape[1],**model_args)
        weights = weights[:3]
        lfunc = outcome_loss
        
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain)
    xtest = df_to_torch(xtest)
    ytrain = [df_to_torch(t) for t in ytrain]
    ytest= [df_to_torch(t) for t in ytest]
    
    model.fit_normalizer(xtrain)
#     normalize = lambda x: (x - xtrain.mean(axis=0)+.01)/(xtrain.std(axis=0)+.01)
#     unnormalize = lambda x: (x * (xtrain.std(axis=0) +.01)) + xtrain.mean(axis=0) - .01
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample)
        loss = lfunc(ytrain,ypred,weights=weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    return model,  best_val_loss, best_loss_metrics

t1_args = {'hidden_layers': [1000], 'dropout': 0.9, 'input_dropout': 0.5}
model1,_,_ = train_state(model_args=t1_args,use_attention=False)
model1

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
epoch 0 train loss 12.584715843200684
val loss 10.469034194946289
______________
epoch 1 train loss 12.27181625366211
val loss 10.260318756103516
______________
epoch 2 train loss 12.172605514526367
val loss 10.057838439941406
______________
epoch 3 train loss 11.924123764038086
val loss 9.861397743225098
______________
epoch 4 train loss 11.583189010620117
val loss 9.670869827270508
______________
epoch 5 train loss 11.3615140914917
val loss 9.486045837402344
______________
epoch 6 train loss 11.173789024353027
val loss 9.306960105895996
______________
epoch 7 train loss 10.843757629394531
val loss 9.13375473022461
______________
epoch 8 train loss 10.620189666748047
val loss 8.966254234313965
______________
epoch 9 train loss 10.544535636901855
val loss 8.804043769836426
______________
epoch 10 train loss 10.400749206542969
val loss 8.647340774536133
______________
epoch 11 train loss 10.13203239440918
val loss 8.496108055114746
______________
epoch 12 train loss 9.8101453781127

epoch 107 train loss 3.637948989868164
val loss 4.158107757568359
______________
epoch 108 train loss 3.4630680084228516
val loss 4.1442413330078125
______________
epoch 109 train loss 3.557813882827759
val loss 4.130547046661377
______________
epoch 110 train loss 3.580655336380005
val loss 4.116995811462402
______________
epoch 111 train loss 3.4690659046173096
val loss 4.10364294052124
______________
epoch 112 train loss 3.410067319869995
val loss 4.090603828430176
______________
epoch 113 train loss 3.45928692817688
val loss 4.077873706817627
______________
epoch 114 train loss 3.374988555908203
val loss 4.065329551696777
______________
epoch 115 train loss 3.426192045211792
val loss 4.05294942855835
______________
epoch 116 train loss 3.4066555500030518
val loss 4.0407304763793945
______________
epoch 117 train loss 3.3791098594665527
val loss 4.028628349304199
______________
epoch 118 train loss 3.4128146171569824
val loss 4.016730308532715
______________
epoch 119 train loss 3.4

epoch 212 train loss 2.6064164638519287
val loss 3.2714107036590576
______________
epoch 213 train loss 2.62460994720459
val loss 3.265824794769287
______________
epoch 214 train loss 2.569301128387451
val loss 3.2602081298828125
______________
epoch 215 train loss 2.6350433826446533
val loss 3.2544400691986084
______________
epoch 216 train loss 2.5891263484954834
val loss 3.248676300048828
______________
epoch 217 train loss 2.568282127380371
val loss 3.242978096008301
______________
epoch 218 train loss 2.5573487281799316
val loss 3.2374496459960938
______________
epoch 219 train loss 2.5776150226593018
val loss 3.2319722175598145
______________
epoch 220 train loss 2.4912197589874268
val loss 3.2265193462371826
______________
epoch 221 train loss 2.6213626861572266
val loss 3.2210707664489746
______________
epoch 222 train loss 2.544618606567383
val loss 3.2155752182006836
______________
epoch 223 train loss 2.531808853149414
val loss 3.2101669311523438
______________
epoch 224 tra

epoch 316 train loss 2.1980443000793457
val loss 2.8349807262420654
______________
epoch 317 train loss 2.2516279220581055
val loss 2.8319449424743652
______________
epoch 318 train loss 2.2116870880126953
val loss 2.8288817405700684
______________
epoch 319 train loss 2.2016820907592773
val loss 2.8259103298187256
______________
epoch 320 train loss 2.2543516159057617
val loss 2.8229711055755615
______________
epoch 321 train loss 2.2158987522125244
val loss 2.820063591003418
______________
epoch 322 train loss 2.3049445152282715
val loss 2.81710147857666
______________
epoch 323 train loss 2.2857961654663086
val loss 2.814126491546631
______________
epoch 324 train loss 2.2143938541412354
val loss 2.811228036880493
______________
epoch 325 train loss 2.2146270275115967
val loss 2.808393955230713
______________
epoch 326 train loss 2.2381787300109863
val loss 2.8056719303131104
______________
epoch 327 train loss 2.2027840614318848
val loss 2.803011655807495
______________
epoch 328 t

...

epoch 616 train loss 1.8102638721466064
val loss 2.357335329055786
______________
epoch 617 train loss 1.8821309804916382
val loss 2.356456756591797
______________
epoch 618 train loss 1.8421550989151
val loss 2.35561203956604
______________
epoch 619 train loss 1.8658998012542725
val loss 2.3548107147216797
______________
epoch 620 train loss 1.8027706146240234
val loss 2.3541066646575928
______________
epoch 621 train loss 1.8508849143981934
val loss 2.3533689975738525
______________
epoch 622 train loss 1.8420522212982178
val loss 2.35263991355896
______________
epoch 623 train loss 1.8534283638000488
val loss 2.3519647121429443
______________
epoch 624 train loss 1.8595632314682007
val loss 2.351236581802368
______________
epoch 625 train loss 1.8404078483581543
val loss 2.350466012954712
______________
epoch 626 train loss 1.7805637121200562
val loss 2.3496806621551514
______________
epoch 627 train loss 1.8499927520751953
val loss 2.3488807678222656
______________
epoch 628 train

epoch 716 train loss 1.8093935251235962
val loss 2.2956342697143555
______________
epoch 717 train loss 1.8172975778579712
val loss 2.295180320739746
______________
epoch 718 train loss 1.8008874654769897
val loss 2.2947776317596436
______________
epoch 719 train loss 1.7355549335479736
val loss 2.294374465942383
______________
epoch 720 train loss 1.7758588790893555
val loss 2.2939488887786865
______________
epoch 721 train loss 1.721895456314087
val loss 2.2934911251068115
______________
epoch 722 train loss 1.8234310150146484
val loss 2.2930240631103516
______________
epoch 723 train loss 1.8571240901947021
val loss 2.29253888130188
______________
epoch 724 train loss 1.7865300178527832
val loss 2.292057991027832
______________
epoch 725 train loss 1.8160970211029053
val loss 2.2916665077209473
______________
epoch 726 train loss 1.7262141704559326
val loss 2.291252851486206
______________
epoch 727 train loss 1.755845308303833
val loss 2.2908432483673096
______________
epoch 728 tr

epoch 816 train loss 1.7169924974441528
val loss 2.2531566619873047
______________
epoch 817 train loss 1.7225542068481445
val loss 2.2527925968170166
______________
epoch 818 train loss 1.7004363536834717
val loss 2.2524161338806152
______________
epoch 819 train loss 1.6965844631195068
val loss 2.2520627975463867
______________
epoch 820 train loss 1.6837445497512817
val loss 2.2517824172973633
______________
epoch 821 train loss 1.728363275527954
val loss 2.2514970302581787
______________
epoch 822 train loss 1.770051121711731
val loss 2.2511847019195557
______________
epoch 823 train loss 1.7271164655685425
val loss 2.2508561611175537
______________
epoch 824 train loss 1.7721513509750366
val loss 2.250488042831421
______________
epoch 825 train loss 1.7059080600738525
val loss 2.2500851154327393
______________
epoch 826 train loss 1.709416389465332
val loss 2.2496907711029053
______________
epoch 827 train loss 1.7152849435806274
val loss 2.249293088912964
______________
epoch 828

epoch 916 train loss 1.6226109266281128
val loss 2.225822925567627
______________
epoch 917 train loss 1.6238603591918945
val loss 2.225560188293457
______________
epoch 918 train loss 1.6393274068832397
val loss 2.22529935836792
______________
epoch 919 train loss 1.6936843395233154
val loss 2.224980115890503
______________
epoch 920 train loss 1.7075833082199097
val loss 2.224660634994507
______________
epoch 921 train loss 1.665897250175476
val loss 2.2242674827575684
______________
epoch 922 train loss 1.6821403503417969
val loss 2.223872661590576
______________
epoch 923 train loss 1.6746023893356323
val loss 2.2234535217285156
______________
epoch 924 train loss 1.7006337642669678
val loss 2.2230160236358643
______________
epoch 925 train loss 1.7257481813430786
val loss 2.222581148147583
______________
epoch 926 train loss 1.6753166913986206
val loss 2.2221100330352783
______________
epoch 927 train loss 1.71686851978302
val loss 2.221709728240967
______________
epoch 928 train 

OutcomeSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=50, out_features=1000, bias=True)
    (1): ReLU()
  )
  (batchnorm): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (nodal_disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (dlt_layers): ModuleList(
    (0): Linear(in_features=1000, out_features=1, bias=True)
    (1): Linear(in_features=1000, out_features=1, bias=True)
    (2): Linear(in_features=1000, out_features=1, bias=True)
    (3): Linear(in_features=1000, out_features=1, bias=True)
    (4): Linear(in_features=1000, out_features=1, bias=True)
    (5): Linear(in_features=1000, out_features=1, bias=True)
    (6): Linear(in_features=1000, out_features=1, bias=True)
    (7): Linear(in_features=10

In [246]:
t2_args = {'hidden_layers': [500, 500], 'dropout': 0.9, 'input_dropout': 0.5}
model2,_,_ = train_state(model_args=t2_args,state=2,use_attention=False)
model2

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
epoch 0 train loss 10.965421676635742
val loss 10.463837623596191
______________
epoch 1 train loss 10.817804336547852
val loss 10.26890754699707
______________
epoch 2 train loss 10.649312019348145


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


val loss 10.082319259643555
______________
epoch 3 train loss 10.486175537109375
val loss 9.902894020080566
______________
epoch 4 train loss 10.31086540222168
val loss 9.730097770690918
______________
epoch 5 train loss 10.197577476501465
val loss 9.562862396240234
______________
epoch 6 train loss 10.046643257141113
val loss 9.400331497192383
______________
epoch 7 train loss 9.90276050567627
val loss 9.241881370544434
______________
epoch 8 train loss 9.725255966186523
val loss 9.087478637695312
______________
epoch 9 train loss 9.60567569732666
val loss 8.936623573303223
______________
epoch 10 train loss 9.475502014160156
val loss 8.789018630981445
______________
epoch 11 train loss 9.335063934326172
val loss 8.64405632019043
______________
epoch 12 train loss 9.241536140441895
val loss 8.501664161682129
______________
epoch 13 train loss 9.122686386108398
val loss 8.361593246459961
______________
epoch 14 train loss 8.989729881286621
val loss 8.223686218261719
______________
epoc

epoch 109 train loss 4.28939962387085
val loss 4.002512454986572
______________
epoch 110 train loss 4.288366317749023
val loss 3.9989430904388428
______________
epoch 111 train loss 4.281261444091797
val loss 3.9953134059906006
______________
epoch 112 train loss 4.279115676879883
val loss 3.9915127754211426
______________
epoch 113 train loss 4.329204559326172
val loss 3.9878389835357666
______________
epoch 114 train loss 4.254798412322998
val loss 3.9845175743103027
______________
epoch 115 train loss 4.236929416656494
val loss 3.9813358783721924
______________
epoch 116 train loss 4.227950096130371
val loss 3.978368043899536
______________
epoch 117 train loss 4.2185235023498535
val loss 3.9753341674804688
______________
epoch 118 train loss 4.216078758239746
val loss 3.9721522331237793
______________
epoch 119 train loss 4.224706172943115
val loss 3.968489646911621
______________
epoch 120 train loss 4.240245819091797
val loss 3.9644663333892822
______________
epoch 121 train los

...

val loss 3.606131076812744
______________
epoch 312 train loss 3.4167873859405518
val loss 3.604841470718384
______________
epoch 313 train loss 3.4165425300598145
val loss 3.603576421737671
______________
epoch 314 train loss 3.410043954849243
val loss 3.6020164489746094
______________
epoch 315 train loss 3.3925907611846924
val loss 3.6006524562835693
______________
epoch 316 train loss 3.4345102310180664
val loss 3.5992469787597656
______________
epoch 317 train loss 3.427387237548828
val loss 3.5977535247802734
______________
epoch 318 train loss 3.411287784576416
val loss 3.5964274406433105
______________
epoch 319 train loss 3.4333302974700928
val loss 3.5948424339294434
______________
epoch 320 train loss 3.4245665073394775
val loss 3.593414545059204
______________
epoch 321 train loss 3.446314573287964
val loss 3.5918984413146973
______________
epoch 322 train loss 3.3924412727355957
val loss 3.5906054973602295
______________
epoch 323 train loss 3.395936965942383
val loss 3.58

val loss 3.524904727935791
______________
epoch 414 train loss 3.2590880393981934
val loss 3.5238914489746094
______________
epoch 415 train loss 3.2677688598632812
val loss 3.5225753784179688
______________
epoch 416 train loss 3.2480385303497314
val loss 3.5213963985443115
______________
epoch 417 train loss 3.25256609916687
val loss 3.520350694656372
______________
epoch 418 train loss 3.30407977104187
val loss 3.5194461345672607
______________
epoch 419 train loss 3.2569167613983154
val loss 3.5187203884124756
______________
epoch 420 train loss 3.264942169189453
val loss 3.518305778503418
______________
epoch 421 train loss 3.277289390563965
val loss 3.5178754329681396
______________
epoch 422 train loss 3.2801218032836914
val loss 3.517397880554199
______________
epoch 423 train loss 3.263296127319336
val loss 3.5169155597686768
______________
epoch 424 train loss 3.261009931564331
val loss 3.5167860984802246
______________
epoch 425 train loss 3.24609375
val loss 3.5168452262878

epoch 518 train loss 3.1524322032928467
val loss 3.4710755348205566
______________
epoch 519 train loss 3.15773606300354
val loss 3.470646858215332
______________
epoch 520 train loss 3.166398525238037
val loss 3.470114231109619
______________
epoch 521 train loss 3.129446268081665
val loss 3.4695284366607666
______________
epoch 522 train loss 3.17154860496521
val loss 3.4688799381256104
______________
epoch 523 train loss 3.1813268661499023
val loss 3.4683172702789307
______________
epoch 524 train loss 3.160810708999634
val loss 3.4679133892059326
______________
epoch 525 train loss 3.17592191696167
val loss 3.4675371646881104
______________
epoch 526 train loss 3.1465444564819336
val loss 3.4671590328216553
______________
epoch 527 train loss 3.1513352394104004
val loss 3.466595411300659
______________
epoch 528 train loss 3.1431989669799805
val loss 3.465945243835449
______________
epoch 529 train loss 3.1617319583892822
val loss 3.4651882648468018
______________
epoch 530 train l

epoch 619 train loss 3.0360467433929443
val loss 3.431000232696533
______________
epoch 620 train loss 3.03600811958313
val loss 3.4303970336914062
______________
epoch 621 train loss 3.0874831676483154
val loss 3.429776430130005
______________
epoch 622 train loss 3.059547185897827
val loss 3.4293129444122314
______________
epoch 623 train loss 3.056511402130127
val loss 3.4288830757141113
______________
epoch 624 train loss 3.0651261806488037
val loss 3.428677558898926
______________
epoch 625 train loss 3.09053635597229
val loss 3.4284474849700928
______________
epoch 626 train loss 3.0810611248016357
val loss 3.4281935691833496
______________
epoch 627 train loss 3.0577433109283447
val loss 3.428241729736328
______________
epoch 628 train loss 3.0967748165130615
val loss 3.428295135498047
______________
epoch 629 train loss 3.0641674995422363
val loss 3.4283084869384766
______________
epoch 630 train loss 3.0535566806793213
val loss 3.428220510482788
______________
epoch 631 train 

OutcomeSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=72, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (disease_layer): Linear(in_features=500, out_features=3, bias=True)
  (nodal_disease_layer): Linear(in_features=500, out_features=3, bias=True)
  (dlt_layers): ModuleList(
    (0): Linear(in_features=500, out_features=1, bias=True)
    (1): Linear(in_features=500, out_features=1, bias=True)
    (2): Linear(in_features=500, out_features=1, bias=True)
    (3): Linear(in_features=500, out_features=1, bias=True)
    (4): Linear(in_features=500, out_features=1, bias=True)
    (5): Linear(in_features=500, out_features=1, bias=True)
    (6): Linear(in_feat

In [247]:
t3_args = {'hidden_layers': [500, 500], 'dropout': 0.9, 'input_dropout': 0.5}
model3,_,_ = train_state(model_args=t3_args,state=3,use_attention=False)
model3

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
epoch 0 train loss 2.133636951446533
val loss 2.132828950881958
______________
epoch 1 train loss 2.129544496536255
val loss 2.109847068786621
______________
epoch 2 train loss 2.1155524253845215
val loss 2.08845853805542
______________
epoch 3 train loss 2.0969343185424805
val loss 2.0688138008117676
______________
epoch 4 train loss 2.1010801792144775
val loss 2.0501344203948975
______________
epoch 5 train loss 2.0887258052825928
val loss 2.0335781574249268
______________
epoch 6 train loss 2.0928192138671875
val loss 2.018505811691284
______________
epoch 7 train loss 2.0904297828674316
val loss 2.004554033279419
______________
epoch 8 train loss 2.0770952701568604
val loss 1.9915950298309326
______________
epoch 9 train loss 2.0776736736297607
val loss 1.9800138473510742
______________
epoch 10 train loss 2.0622060298919678
val loss 1.9694223403930664
______________
epoch 11 train loss 2.0624523162841797
val loss 1.9600071907043457
______________
epoch 12 train loss 2.0605840

val loss 1.5188090801239014
______________
epoch 104 train loss 1.7023874521255493
val loss 1.5162739753723145
______________
epoch 105 train loss 1.7149555683135986
val loss 1.5139126777648926
______________
epoch 106 train loss 1.7155201435089111
val loss 1.5123188495635986
______________
epoch 107 train loss 1.7003014087677002
val loss 1.5108778476715088
______________
epoch 108 train loss 1.6958699226379395
val loss 1.509464979171753
______________
epoch 109 train loss 1.7166173458099365
val loss 1.5081900358200073
______________
epoch 110 train loss 1.700044870376587
val loss 1.5071276426315308
______________
epoch 111 train loss 1.6867841482162476
val loss 1.5060707330703735
______________
epoch 112 train loss 1.6948527097702026
val loss 1.5049229860305786
______________
epoch 113 train loss 1.6787575483322144
val loss 1.5035303831100464
______________
epoch 114 train loss 1.6999809741973877
val loss 1.5020933151245117
______________
epoch 115 train loss 1.6964943408966064
val lo

EndpointSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=70, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (outcome_layer): Linear(in_features=500, out_features=3, bias=True)
)

In [156]:
def gridsearch_attention_transition_models(state=1,attentions=[True,False]):
    model_arglist = [
        {
            'hidden_layers': [100],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [500],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [10],
        },
        {
            'hidden_layers': [100],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [500],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [5],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [10,10],
        },
        {
            'hidden_layers': [500,500],
            'attention_heads': [10,10],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [5,5],
        },
        {
            'hidden_layers': [500,500],
            'attention_heads': [5,5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for use_attention in attentions:
            embed_sizes = [200,400] if use_attention else [0]
            for embed_size in embed_sizes:
                args['embed_size'] = embed_size
                for dropout in [.9,]:
                    args['dropout'] = dropout
                    for input_dropout in [.5]:
                        args['input_dropout'] = input_dropout
                    
                        model,m_loss,m_metrics = train_state(model_args=args,state=state,use_attention=use_attention,verbose=False)
                        print('done',k,m_loss)
                        k+=1
                        if m_loss < best_loss:
                            best_loss = m_loss
                            best_metrics  = m_metrics
                            best_model = model
                            best_args = args
                            print('_++++++++++New Best++++____')
                            print(best_loss)
                            print(best_metrics)
                            print(best_args)
                            print(use_attention)
                            print('___________')
                            print('++++++++')
                            print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model
# model1 = gridsearch_attention_transition_models(1)
# model1

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.764965534210205 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6839781628293378, 'roc_macro': 0.6330226813097114}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.9510489510489512, 0.9313131313131313, 0.9527363184079602, -1, 0.9092198581560285, -1, 0.9333333333333333, -1], 'auc_mean': 0.20970644903242558}}
done 0 2.764965534210205
_++++++++++New Best++++____
2.764965534210205
{'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6839781628293378, 'roc_macro': 0.6330226813097114}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6238666993383974, 'roc_macro': 0.6753406617537051}, 'mod': {'accuracy': 0.333333333333

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 4.025295257568359 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6755281272252552, 'roc_macro': 0.6340827355168788}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.5679000245037981, 'roc_macro': 0.559701470027557}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.5679000245037981, 'roc_macro': 0.559701470027557}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.766899766899767, 0.8424242424242425, 0.7170398009950248, -1, 0.75177304964539, -1, 0.8880952380952382, -1], 'auc_mean': 0.12077901225745781}}
done 1 4.025295257568359


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.7161152362823486 {'pd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6835509138381202, 'roc_macro': 0.6361547246381728}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6163195295270767, 'roc_macro': 0.6741712393886307}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.6163195295270767, 'roc_macro': 0.6741712393886307}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.9137529137529138, 0.9683501683501683, 0.9359452736318408, -1, 0.9304964539007092, -1, 0.9011904761904762, -1], 'auc_mean': 0.20621691072826354}}
done 28 2.7161152362823486


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 2.3858258724212646 {'pd': {'accuracy': 0.4969654528478058, 'roc_micro': 0.6851174934725849, 'roc_macro': 0.6237181607808212}, 'nd': {'accuracy': 0.5198805594845198, 'roc_micro': 0.7011026709139917, 'roc_macro': 0.6828778105952019}, 'mod': {'accuracy': 0.5198805594845198, 'roc_micro': 0.7011026709139917, 'roc_macro': 0.6828778105952019}, 'dlts': {'accuracy': [0.9794520547945206, 0.9246575342465754, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.958904109589041, 1.0], 'accuracy_mean': 0.9683219178082192, 'auc': [0.8951048951048951, 0.9555555555555556, 0.9253731343283582, -1, 0.9432624113475176, -1, 0.8952380952380953, -1], 'auc_mean': 0.20181676144680272}}
done 29 2.3858258724212646
_________
+++++++++++
best stuff 2.251129150390625
{'pd': {'accuracy': 0.375, 'roc_micro': 0.6783764538333729, 'roc_macro': 0.6248571736860679}, 'nd': {'accuracy': 0.4801980198019802, 'roc_micro': 0.6757167360940945, 'roc_macro': 0.6780191426930556}, 'mod': {'accuracy': 0.4801980198019802,

OutcomeAttentionSimulator(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=100, out_features=1000, bias=True)
  )
  (batchnorm): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (resize_layer): Linear(in_features=63, out_features=100, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  )
  (final_layer): Linear(in_features=1000, out_features=3, bias=True)
  (activation): ReLU()
  (disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (nodal_disease_layer): Linear(in_features=1000, out_features=3, bias=True)
  (dlt_layers): ModuleList(
    (0): Linear(in_features=1000, out_features=1

In [157]:
# model2 = gridsearch_atten/.tion_transition_models(2)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains clas

best loss 3.808809757232666 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9072182055232902, 'roc_macro': -1}, 'nd': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'mod': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.2564102564102564, 0.5926616915422885, 0.60431654676259, -1, 0.5035460992907801, -1, 0.5943262411347519, -1], 'auc_mean': -0.05609239560741665}}
done 0 3.808809757232666
_++++++++++New Best++++____
3.808809757232666
{'pd': {'accuracy': 0.5, 'roc_micro': 0.9072182055232902, 'roc_macro': -1}, 'nd': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.47419713120932633}, 'mod': {'accuracy': 0.350210970464135, 'roc_micro': 0.7220952335121871, 'roc_macro': 0.474197131209326

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

best loss 3.8991506099700928 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9260874718501837, 'roc_macro': -1}, 'nd': {'accuracy': 0.32383966244725737, 'roc_micro': 0.7193041553379371, 'roc_macro': 0.484346802563266}, 'mod': {'accuracy': 0.32383966244725737, 'roc_micro': 0.7193041553379371, 'roc_macro': 0.484346802563266}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.48717948717948717, 0.5690298507462687, 0.4696813977389517, -1, 0.5397163120567376, -1, 0.3588652482269504, -1], 'auc_mean': -0.07194096300645057}}
done 28 3.8991506099700928


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains clas

best loss 3.4415688514709473 {'pd': {'accuracy': 0.4959349593495935, 'roc_micro': 0.9374659239066019, 'roc_macro': -1}, 'nd': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'mod': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'dlts': {'accuracy': [0.9794520547945206, 0.9178082191780822, 0.952054794520548, 1.0, 0.9657534246575342, 1.0, 0.9657534246575342, 1.0], 'accuracy_mean': 0.9726027397260274, 'auc': [0.3776223776223776, 0.595771144278607, 0.6402877697841727, -1, 0.6624113475177306, -1, 0.3801418439716312, -1], 'auc_mean': -0.04297068960318512}}
done 29 3.4415688514709473
_++++++++++New Best++++____
3.4415688514709473
{'pd': {'accuracy': 0.4959349593495935, 'roc_micro': 0.9374659239066019, 'roc_macro': -1}, 'nd': {'accuracy': 0.34236550632911394, 'roc_micro': 0.7523639950915522, 'roc_macro': 0.6504770843566575}, 'mod': {'accuracy': 0.34236550632911394, 'roc_micro': 0.75236399

In [158]:
# model3 = gridsearch_attention_transition_models(3)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.9533352851867676 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.22294936, 'auc': 0.61875}, 'FT': {'accuracy': -1, 'mse': 0.22988118, 'auc': 0.6208881578947368}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.23439899, 'auc': 0.7096153846153845}}
done 0 1.9533352851867676
_++++++++++New Best++++____
1.9533352851867676
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.22294936, 'auc': 0.61875}, 'FT': {'accuracy': -1, 'mse': 0.22988118, 'auc': 0.6208881578947368}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.23439899, 'auc': 0.7096153846153845}}
{'hidden_layers': [100], 'attention_heads': [1], 'embed_size': 200, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.9169039726257324 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.23428532, 'auc': 0.5783653846153846}, 'FT': {'accuracy': -1, 'mse': 0.21831562, 'auc': 0.6165021929824561}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.21789795, 'auc': 0.7125}}
done 1 1.9169039726257324
_++++++++++New Best++++____
1.9169039726257324
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.23428532, 'auc': 0.5783653846153846}, 'FT': {'accuracy': -1, 'mse': 0.21831562, 'auc': 0.6165021929824561}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.21789795, 'auc': 0.7125}}
{'hidden_layers': [100], 'attention_heads': [1], 'embed_size': 400, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


...

here
best loss 1.513304352760315 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.15192407, 'auc': 0.6110576923076922}, 'FT': {'accuracy': -1, 'mse': 0.19458365, 'auc': 0.5923793859649122}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14482027, 'auc': 0.7080128205128204}}
done 27 1.513304352760315


/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.455763339996338 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
done 28 1.455763339996338
_++++++++++New Best++++____
1.455763339996338
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
{'hidden_layers': [500, 500], 'attention_heads': [5, 5], 'embed_size': 400, 'dropout': 0.9, 'input_dropout': 0.5}
True
___________
++++++++



/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
best loss 1.4676107168197632 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12858523, 'auc': 0.6384615384615384}, 'FT': {'accuracy': -1, 'mse': 0.19938947, 'auc': 0.6077302631578947}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.13879225, 'auc': 0.7153846153846154}}
done 29 1.4676107168197632
_________
+++++++++++
best stuff 1.455763339996338
{'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.12546952, 'auc': 0.6596153846153847}, 'FT': {'accuracy': -1, 'mse': 0.20594049, 'auc': 0.6058114035087719}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.14230496, 'auc': 0.7096153846153845}}
{'hidden_layers': [500, 500], 'attention_heads': [5, 5], 'embed_size': 0, 'dropout': 0.9, 'input_dropout': 0.5}


In [ ]:
def load_trained_models():
    files = Const.tuned_transition_models
    decision_file = Const.tuned_decision_model
    [model1,model2,model3] = [torch.load(file) for file in files]
    decision_model = torch.load(decision_file)
    return decision_model, model1,model2,model3
_, model1, model2, model3 =load_trained_models()

In [248]:

def shuffle_col(v,col=None):
    if col is None:
        col = np.random.choice([i for i in range(v.shape[1])])
    idx = torch.randperm(v.shape[0])
    vv = torch.clone(v)
    vv[:,col] = vv[idx,col]
    return vv
    
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=100,
    weights=[3,1,1], #realtive weight of survival, feeding tube, and aspiration
    imitation_weight=1,
    shufflecol_chance = 0.1,
    reward_weight=10,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_attention=False,
    verbose=True,
    threshold_decisions=True,
    use_smote=True,
    **model_kwargs,
):
    
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()

    data = dataset.processed_df.copy()
    
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids])
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    def outcome_loss(ypred):
        #convert survival to death
        loss = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),weights[0])
        for i,weight in enumerate(weights[1:]):
            newloss = torch.mean(ypred[:,i])*weight
            loss = torch.add(loss,torch.mul(newloss,weight))
        return loss
    
    mse = torch.nn.MSELoss()
    nllloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,.5).type(torch.FloatTensor)

    
    #save the inputs from the whole dataset for future reference
    full_data = []
    for mstep in [0,1,2]:
        full_data_step = [baseline, get_dlt(min(mstep,1)),
                     get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
        full_data_step = torch.cat([formatdf(fd,train_ids+test_ids) for fd in full_data_step],axis=1)
        full_data.append(full_data_step)
    full_data = torch.stack(full_data)
    model.save_memory(full_data)

    def step(train=True):
        if train:
            model.train(True)
            optimizer.zero_grad()
            ids = train_ids
        else:
            ids = test_ids
            model.eval()
            
            
        ytrain = df_to_torch(outcomedf.loc[ids])
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        o1 = model(torch.cat(xxtrain,axis=1),position=0)
            
        decision1_imitation = o1[:,3]
        decision1 = o1[:,0]
        if threshold_decisions:
            decision1 = threshold(decision1)
#         imitation_loss1 = bce(threshold(decision1_imitation),ytrain[:,0])
        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1)[:,4]
        
#         imitation_loss2 =  bce(threshold(decision2_imitation),ytrain[:,1])
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
#         if model.keys is None:
#             keyvals = torch.cat([formatdf(xxx,train_ids+test_ids) for xxx in x2_imitation],axis=1)
#             model.set_keys(keyvals)
            
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        
        decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2)[:,5]
        
#         imitation_loss3 = bce(threshold(decision3_imitation),ytrain[:,2])
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)
        if train and shufflecol_chance > 0.0001:
            for col in range(baseline_train_base.shape[1]): 
                if np.random.random() < shufflecol_chance:
                    baseline_train = shuffle_col(baseline_train,col)
                    
        
        xi1 = torch.cat([xx1,decision1.view(-1,1)],axis=1)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)
        #this outputs log likelihoods (except for dlts) -> convert to probability
        ypd1 = torch.exp(ypd1)
        ynd1 = torch.exp(ynd1)
        ymod = torch.exp(ymod)
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        
        decision2 = model(torch.cat(x1,axis=1),position=1)[:,1] 
        if threshold_decisions:
            decision2 = threshold(decision2)
            
        xi2 = torch.cat([xx2,decision1.view(-1,1),decision2.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)
        ypd2 = torch.exp(ypd2)
        ynd2 = torch.exp(ynd2)
        ycc = torch.exp(ycc)
        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
            
        decision3 = model(torch.cat(x2,axis=1),position=2)[:,2]
        if threshold_decisions:
            decision3 = threshold(decision3)
            
        
        xi3 = torch.cat([xx3,decision1.view(-1,1),decision2.view(-1,1),decision3.view(-1,1)],axis=1)
        outcomes = tmodel3(xi3)

        reward_loss = outcome_loss(outcomes)
        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss]
        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            for i,decision in enumerate([decision1_imitation,decision2_imitation,decision3_imitation]):
                dec = decision.cpu().detach().numpy()
                dec0 = (dec > .5).astype(int)
                out = ytrain[:,i].cpu().detach().numpy()
                acc = accuracy_score(out,dec > .5)
                auc = roc_auc_score(out,dec)
                scores.append({'decision': i,'accuracy': acc,'auc': auc})
            return losses, scores
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics = step(False)
        vl = val_losses[0] + val_losses[1]
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('train imitation',losses[0].item(),'reward',losses[1].item())
            print('val imitation',val_losses[0].item(),'reward',val_losses[1].item())
            print('val loss',vl.item(),best_val_loss.item())
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss

from Models import *
args = {
    'hidden_layers': [500], 
    'attention_heads': [2], 
    'embed_size': 210, 
    'dropout': 0.9, 
    'input_dropout': 0.5, 
    'shufflecol_chance': 0.25,
}
decision_model, _, _ = train_decision_model(model1,model2,model3,lr=.0001,use_attention=True,**args)

/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.
  
/home/evl/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (55) have mixed types.Specify dtype option on import or set low_memory=False.


here
______epoch 0 _____
train imitation 2.211329936981201 reward 2.4836373329162598
val imitation 2.1203558444976807 reward 1.8343467712402344
val loss 3.954702615737915 1000000000.0
[{'decision': 0, 'accuracy': 0.4589041095890411, 'auc': 0.49615384615384617}, {'decision': 1, 'accuracy': 0.410958904109589, 'auc': 0.4383223684210526}, {'decision': 2, 'accuracy': 0.3835616438356164, 'auc': 0.5884615384615385}]
______epoch 1 _____
train imitation 2.2313594818115234 reward 2.487095355987549
val imitation 2.111082077026367 reward 1.8335179090499878
val loss 3.9446001052856445 3.954702615737915
[{'decision': 0, 'accuracy': 0.4726027397260274, 'auc': 0.5}, {'decision': 1, 'accuracy': 0.3972602739726027, 'auc': 0.4429824561403509}, {'decision': 2, 'accuracy': 0.3904109589041096, 'auc': 0.5958333333333333}]
______epoch 2 _____
train imitation 2.18521785736084 reward 2.4834415912628174
val imitation 2.103510618209839 reward 1.8342481851577759
val loss 3.9377589225769043 3.9446001052856445
[{'de

______epoch 21 _____
train imitation 2.087005615234375 reward 2.4813249111175537
val imitation 1.9872283935546875 reward 1.8340314626693726
val loss 3.8212599754333496 3.8253471851348877
[{'decision': 0, 'accuracy': 0.6917808219178082, 'auc': 0.5423076923076924}, {'decision': 1, 'accuracy': 0.678082191780822, 'auc': 0.6373355263157895}, {'decision': 2, 'accuracy': 0.7054794520547946, 'auc': 0.7384615384615385}]
______epoch 22 _____
train imitation 2.08001446723938 reward 2.4827895164489746
val imitation 1.9833742380142212 reward 1.8335046768188477
val loss 3.8168787956237793 3.8212599754333496
[{'decision': 0, 'accuracy': 0.6917808219178082, 'auc': 0.5427884615384615}, {'decision': 1, 'accuracy': 0.6917808219178082, 'auc': 0.6417214912280702}, {'decision': 2, 'accuracy': 0.7054794520547946, 'auc': 0.7397435897435898}]
______epoch 23 _____
train imitation 2.080533504486084 reward 2.483095645904541
val imitation 1.9794373512268066 reward 1.8330190181732178
val loss 3.8124563694000244 3.8

______epoch 41 _____
train imitation 2.0078206062316895 reward 2.4788262844085693
val imitation 1.9121582508087158 reward 1.8314732313156128
val loss 3.743631362915039 3.7483153343200684
[{'decision': 0, 'accuracy': 0.6986301369863014, 'auc': 0.5418269230769232}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.6970942982456141}, {'decision': 2, 'accuracy': 0.726027397260274, 'auc': 0.7567307692307692}]
______epoch 42 _____
train imitation 2.016968250274658 reward 2.4794843196868896
val imitation 1.906798243522644 reward 1.8312925100326538
val loss 3.738090753555298 3.743631362915039
[{'decision': 0, 'accuracy': 0.7123287671232876, 'auc': 0.5413461538461538}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.6987390350877194}, {'decision': 2, 'accuracy': 0.7328767123287672, 'auc': 0.7564102564102564}]
______epoch 43 _____
train imitation 2.022209644317627 reward 2.4815211296081543
val imitation 1.9016683101654053 reward 1.8311307430267334
val loss 3.7327990531921387 3.7380

______epoch 61 _____
train imitation 1.9529986381530762 reward 2.480656385421753
val imitation 1.7958595752716064 reward 1.8275448083877563
val loss 3.6234045028686523 3.629100799560547
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.5307692307692309}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.7302631578947368}, {'decision': 2, 'accuracy': 0.7671232876712328, 'auc': 0.7689102564102565}]
______epoch 62 _____
train imitation 1.9598884582519531 reward 2.4786102771759033
val imitation 1.7911829948425293 reward 1.827378511428833
val loss 3.6185615062713623 3.6234045028686523
[{'decision': 0, 'accuracy': 0.7602739726027398, 'auc': 0.5298076923076923}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.7299890350877193}, {'decision': 2, 'accuracy': 0.7671232876712328, 'auc': 0.7711538461538462}]
______epoch 63 _____
train imitation 1.9693411588668823 reward 2.4784295558929443
val imitation 1.7863849401474 reward 1.8270325660705566
val loss 3.613417625427246 3.61

...

______epoch 421 _____
train imitation 1.4907805919647217 reward 2.48705792427063
val imitation 1.367345929145813 reward 1.8151280879974365
val loss 3.182474136352539 3.1494641304016113
[{'decision': 0, 'accuracy': 0.773972602739726, 'auc': 0.5052884615384615}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7264254385964912}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.7996794871794872}]
______epoch 422 _____
train imitation 1.4473872184753418 reward 2.4878182411193848
val imitation 1.368731141090393 reward 1.8146554231643677
val loss 3.1833865642547607 3.1494641304016113
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.5043269230769231}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7258771929824561}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.7987179487179488}]
______epoch 423 _____
train imitation 1.4611220359802246 reward 2.4862561225891113
val imitation 1.3684329986572266 reward 1.8146554231643677
val loss 3.1830883026123047 3.14

______epoch 441 _____
train imitation 1.452100157737732 reward 2.485997438430786
val imitation 1.3591127395629883 reward 1.815011978149414
val loss 3.1741247177124023 3.1494641304016113
[{'decision': 0, 'accuracy': 0.773972602739726, 'auc': 0.5067307692307692}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7272478070175439}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.7977564102564103}]
______epoch 442 _____
train imitation 1.4558651447296143 reward 2.4872870445251465
val imitation 1.3615901470184326 reward 1.815011978149414
val loss 3.1766021251678467 3.1494641304016113
[{'decision': 0, 'accuracy': 0.773972602739726, 'auc': 0.50625}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7269736842105263}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.7977564102564103}]
______epoch 443 _____
train imitation 1.43950617313385 reward 2.483926773071289
val imitation 1.3617024421691895 reward 1.8151274919509888
val loss 3.1768298149108887 3.14946413040161

______epoch 461 _____
train imitation 1.4180757999420166 reward 2.4864275455474854
val imitation 1.3613919019699097 reward 1.815321683883667
val loss 3.176713466644287 3.1494641304016113
[{'decision': 0, 'accuracy': 0.773972602739726, 'auc': 0.5057692307692309}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7275219298245614}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.8006410256410257}]
______epoch 462 _____
train imitation 1.4571363925933838 reward 2.4854960441589355
val imitation 1.3703134059906006 reward 1.8152425289154053
val loss 3.185555934906006 3.1494641304016113
[{'decision': 0, 'accuracy': 0.7602739726027398, 'auc': 0.5019230769230769}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7266995614035088}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.8006410256410257}]
______epoch 463 _____
train imitation 1.441050410270691 reward 2.487823724746704
val imitation 1.3735852241516113 reward 1.8152425289154053
val loss 3.1888277530670166 

______epoch 481 _____
train imitation 1.400996446609497 reward 2.483564853668213
val imitation 1.363287329673767 reward 1.8152425289154053
val loss 3.178529739379883 3.1494641304016113
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.5076923076923077}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7308114035087718}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.798397435897436}]
______epoch 482 _____
train imitation 1.3719781637191772 reward 2.4844346046447754
val imitation 1.363943099975586 reward 1.8151441812515259
val loss 3.1790871620178223 3.1494641304016113
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.5072115384615384}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7308114035087719}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.7987179487179487}]
______epoch 483 _____
train imitation 1.3932750225067139 reward 2.486706018447876
val imitation 1.3636590242385864 reward 1.815140962600708
val loss 3.178800106048584 3.14

______epoch 501 _____
train imitation 1.3956347703933716 reward 2.485196352005005
val imitation 1.3496893644332886 reward 1.8147412538528442
val loss 3.164430618286133 3.1494641304016113
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.5072115384615385}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7310855263157895}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.801923076923077}]
______epoch 502 _____
train imitation 1.4084450006484985 reward 2.484103202819824
val imitation 1.3517454862594604 reward 1.814578890800476
val loss 3.1663243770599365 3.1494641304016113
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.5076923076923078}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7305372807017544}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.8012820512820513}]
______epoch 503 _____
train imitation 1.3505301475524902 reward 2.485971689224243
val imitation 1.3511030673980713 reward 1.8145538568496704
val loss 3.1656570434570312 

In [66]:
def gridsearch_decision_model(m1,m2,m3):
    model_arglist = [

        {
            'hidden_layers': [100,100],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [50,50],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [2,2],
        },
        {
            'hidden_layers': [300],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [600],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [300,300],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [600,600],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000],
            'attention_heads': [5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for embed_size in [0,120,210]:
            #embed_size = 0 skips the firt layer that makes the sizes right
            if embed_size == 0 and args['attention_heads'][0] != 1:
                continue
            args['embed_size'] = embed_size
            for dropout in [.5,.9]:
                args['dropout'] = dropout
                for input_dropout in [.5]:
                    args['input_dropout'] = input_dropout
                    for shufflecol_chance in [.1,.5]:
                        args['shufflecol_chance'] = shufflecol_chance
                        model,m_metrics,m_loss = train_decision_model(m1,m2,m3,use_attention=True,verbose=False,**args)
                        print('done',k,m_loss)
                        print('curr best',best_loss)
                        k+=1
                        if m_loss < best_loss:
                            best_loss = m_loss
                            best_metrics  = m_metrics
                            best_model = model
                            best_args = args
                            print('_++++++++++New Best++++____')
                            print(best_loss)
                            print(best_metrics)
                            print(best_args)
                            print('___________')
                            print('++++++++')
                            print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model

from Models import *
decision_model = gridsearch_decision_model(model1,model2,model3)
decision_model

/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.7922, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6634615384615385}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6461074561403509}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.728525641025641}]
done 0 tensor(2.7922, grad_fn=<AddBackward0>)
curr best 100000000000
_++++++++++New Best++++____
tensor(2.7922, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6634615384615385}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6461074561403509}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.728525641025641}]
{'hidden_layers': [100, 100], 'attention_heads': [1, 1], 'embed_size': 0, 'dropout': 0.5, 'input_dropout': 0.5, 'shufflecol_chance': 0.1}
___________
++++++++



/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.8300, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6725961538461539}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6639254385964912}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7371794871794872}]
done 1 tensor(2.8300, grad_fn=<AddBackward0>)
curr best tensor(2.7922, grad_fn=<AddBackward0>)


/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

...

/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.7552, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.68125}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.7044956140350876}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7625}]
done 82 tensor(2.7552, grad_fn=<AddBackward0>)
curr best tensor(2.6872, grad_fn=<AddBackward0>)


/data/DigitalTwinVis/python/Preprocessing.py:182: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:189: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/data/DigitalTwinVis/python/Preprocessing.py:203: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.means = self.processed_df.mean(axis=0)
/data/DigitalTwinVis/python/Preprocessing.py:204: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or speci

++++++++++Final+++++++++++
best tensor(2.8200, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6600961538461538}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.6880482456140351}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.7612179487179487}]
done 83 tensor(2.8200, grad_fn=<AddBackward0>)
curr best tensor(2.6872, grad_fn=<AddBackward0>)
_________
+++++++++++
best stuff tensor(2.6872, grad_fn=<AddBackward0>)
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.6817307692307693}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.7075109649122806}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.7807692307692308}]
{'hidden_layers': [600], 'attention_heads': [3], 'embed_size': 210, 'dropout': 0.9, 'input_dropout': 0.5, 'shufflecol_chance': 0.5}


DecisionAttentionModel(
  (input_dropout): Dropout(p=0.5, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=210, out_features=600, bias=True)
  )
  (batchnorm): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.9, inplace=False)
  (sigmoid): Sigmoid()
  (softmax): LogSoftmax(dim=1)
  (final_layer): Linear(in_features=600, out_features=6, bias=True)
  (resize_layer): Linear(in_features=97, out_features=210, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=210, out_features=210, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((210,), eps=1e-05, elementwise_affine=True)
  )
  (activation): ReLU()
)

In [249]:
torch.save(decision_model,'../data/models/final_decision_model_' + decision_model.identifier + '.pt')
print('../data/models/final_decision_model_' + decision_model.identifier + '.pt')

../data/models/final_decision_model_statedecisions_input119_dims500_dropout0.5,0.9.pt


In [250]:
torch.save(model1,'../data/models/final_transition1_model_' + model1.identifier + '.pt')
torch.save(model2,'../data/models/final_transition2_model_' + model2.identifier + '.pt')
torch.save(model3,'../data/models/final_outcome_model_' + model3.identifier + '.pt')
print('../data/models/final_transition1_model_' + model1.identifier + '.pt')
print('../data/models/final_transition2_model_' + model2.identifier + '.pt')
print('../data/models/final_outcome_model_' + model3.identifier + '.pt')

../data/models/final_transition1_model_state1_input50_dims1000_dropout0.5,0.9.pt
../data/models/final_transition2_model_state2_input72_dims500,500_dropout0.5,0.9.pt
../data/models/final_outcome_model_state1_input70_dims500,500_dropout0.5,0.9.pt


In [ ]:
xatt = []
for att,xxdf in zip(list(attributions),xdf):
    new = pd.DataFrame(att.cpu().detach().numpy(),columns=xxdf.columns,index=xxdf.index)
    xatt.append(new)
attributions = pd.concat(xatt,axis=1)
attributions

In [ ]:
attributions.sum().sort_values()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.subplots(1,1,figsize=(100,100))
sns.heatmap(data=attributions.T,ax=fig[1])

In [ ]:
def breakup_state_models(state_model):
    #for state 1 and 2
    models = {}
    models['pd'] = lambda x: state_model(x)[0]
    models['nd'] = lambda x: state_model(x)[1]
    models['chemo'] = lambda x: state_model(x)[2]
    for i,dlt in enumerate(Const.dlt1):
        models[dlt] = lambda x: state_model(x)[3][:,i]
    return models

def breakup_outcome_models(omodel):
    models = {}
    for i,name in enumerate(Const.outcomes):
        models[name] = lambda x: omodel(x)[:,i].reshape(-1,1)
    return models

def get_all_models(m1,m2,m3):
    state1_models = breakup_state_models(m1)
    state2_models = breakup_state_models(m2)
    state3_models = breakup_outcome_models(m3)
    all_models = {}
    for i,sm in enumerate([state1_models,state2_models,state3_models]):
        for ii,m in sm.items():
            all_models[ii +  '_state' + str(i+1)] = m
    return all_models

all_models = get_all_models(model,model2,model3)
all_models

In [ ]:

def get_ytrue(name,df):
    outcomes=None
    value = None
    if name == 'pd_state1':
        outcomes = df[Const.primary_disease_states]
    elif name == 'pd_state2':
        outcomes = df[Const.primary_disease_states2]
    elif name == 'nd_state1':
        outcomes = df[Const.nodal_disease_states]
    elif name == 'nd_state2':
        outcomes = df[Const.nodal_disease_states2]
    elif name == 'chemo_state1':
        outcomes = df[Const.modifications]
    elif name == 'chemo_state2':
        outcomes = df[Const.ccs]   
    if outcomes is not None:
        value = outcomes.idxmax(axis=1)
    if 'DLT' in name:
        newname = name.replace('_state', ' ').replace('1','').strip()
        value = df[newname]
    if name.replace('_state3','') in Const.outcomes:
        value = df[name.replace('_state3','')]
    if value is None:
        print(name,df.columns)
    return value

def check_impact_of_decisions(model_dict,data):
    results = []
    #todo: this is wrong fix it
    ids = []
    df = data.get_data()
    outcomedict = {step: pd.concat(data.get_intermediate_outcomes(step=step),axis=1) for step in [1,2,3]}
    for decision in Const.decisions:
        for name, model in model_dict.items():
            step = int(name[-1])
            subset0 = dataset.get_input_state(step=step,fixed={decision: 0})
            subset1 = dataset.get_input_state(step=step,fixed={decision: 1})
            outcomes = outcomedict[step]
            ids = subset0.index.values
            x0 = df_to_torch(subset0)
            x1 = df_to_torch(subset1)
            y0 = model(x0).detach().cpu().numpy()
            y1 = model(x1).detach().cpu().numpy()
            original = data.get_input_state(step=step)
            xx = df_to_torch(original)
            yy = model(xx).detach().cpu().numpy()
            ytrue = get_ytrue(name,outcomes)
            if "DLT" in name:
                y0 = y0.argmax(axis=1).reshape(-1,1)
                y1 = y1.argmax(axis=1).reshape(-1,1)
                yy = yy.argmax(axis=1).reshape(-1,1)
                change = y0 - y1
                decision_change = (y0 != y1).astype(int)
            elif y0.shape[1] == 1:
                change = y1 - y0
                decision_change = np.abs((y0 > .5).astype(int) - (y1 > .5).astype(int))
            else:
                index = np.unravel_index(np.argmax(yy, axis=1), yy.shape)
                change = (y0[index] - y1[index]).reshape(-1,1)
                decision_change =  (y0.argmax(axis=1).reshape(-1,1) != y1.argmax(axis=1).reshape(-1,1)).astype(int)
                yy = yy.argmax(axis=1).reshape(-1,1)
                y1 = y1.argmax(axis=1).reshape(-1,1)
                y0 = y0.argmax(axis=1).reshape(-1,1)
            outcome = name.replace('_state','')
            for ii,pid in enumerate(ids):
                oo = ytrue.loc[pid]
                onew = y0[ii][0]
                original_decision = df.loc[pid,decision]
                if original_decision > 0:
                    onew = y0[ii][0]
                oname = Const.name_dict.get(name)
                if oname is not None:
                    onew = oname[onew]
                entry = {'id': pid, 'decision': decision,'outcome': outcome,'original_choice': original_decision, 'original_result': oo, 'alt_result': onew, 'change': change[ii][0], 'decision_change': decision_change[ii][0]}
                results.append(entry)
    return pd.DataFrame(results)

test = check_impact_of_decisions(all_models,dataset)
test

In [ ]:
data.get_data()['SD Primary 2'].sum()

In [ ]:
(test[test.outcome == 'pd2'].original_result == 'SD Primary 2').sum()

In [ ]:
check_impact_of_decisions(all_models,dataset).to_csv('../data/decision_impacts.csv')